# Semantic Kernel


### Import and Initialization

In [5]:
import semantic_kernel as sk

kernel = sk.Kernel()

In [6]:
# Add OpenAI Text Completion to kernel
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))

In [7]:
# Running a semantic kernel
# Wrap your prompt in a function
prompt = kernel.create_semantic_function("""
1) A robot may not injure a human being or, through inaction,
allow a human being to come to harm.

2) A robot must obey orders given it by human beings except where
such orders would conflict with the First Law.

3) A robot must protect its own existence as long as such protection
does not conflict with the First or Second Law.

Give me the TLDR in exactly 5 words.""")

# Run your prompt
print(prompt()) # => Robots must not harm humans.



Robots must not harm humans.


**Semantic Functions are prompt with Input params**

In [8]:
# Create a reusable function with one input parameter
summarize = kernel.create_semantic_function("{{$input}}\n\nOne line TLDR with the fewest words.")

# Summarize the laws of thermodynamics
print(summarize("""
1st Law of Thermodynamics - Energy cannot be created or destroyed.
2nd Law of Thermodynamics - For a spontaneous process, the entropy of the universe increases.
3rd Law of Thermodynamics - A perfect crystal at zero Kelvin has zero entropy."""))

# Summarize the laws of motion
print(summarize("""
1. An object at rest remains at rest, and an object in motion remains in motion at constant speed and in a straight line unless acted on by an unbalanced force.
2. The acceleration of an object depends on the mass of the object and the amount of force applied.
3. Whenever one object exerts a force on another object, the second object exerts an equal and opposite on the first."""))

# Summarize the law of universal gravitation
print(summarize("""
Every point mass attracts every single other point mass by a force acting along the line intersecting both points.
The force is proportional to the product of the two masses and inversely proportional to the square of the distance between them."""))

# Output:
# > Energy conserved, entropy increases, zero entropy at 0K.
# > Objects move in response to forces.
# > Gravitational force between two point masses is inversely proportional to the square of the distance between them.


Energy conserved, entropy increases, zero entropy at 0K.

Objects move in response to forces.


Gravitational force between two point masses is inversely proportional to the square of the distance between them.


**Using Skills**

In [12]:
skill = kernel.import_semantic_skill_from_directory("../semantic-kernel/samples/skills/", "FunSkill")
joke_function = skill["Joke"]

print(joke_function("time travel to dinosaur age"))


A time traveler went back to the dinosaur age and was amazed by the size of the creatures. He asked one of the dinosaurs, "How do you manage to get around with such short legs?"

The dinosaur replied, "It's easy, I just take my time!"


**Inline Semantic function**

In [13]:
prompt = """{{$input}}
Summarize the content above.
"""

summarize = kernel.create_semantic_function(prompt, max_tokens=2000, temperature=0.2, top_p=0.5)

In [14]:
input_text = """
Demo (ancient Greek poet)
From Wikipedia, the free encyclopedia
Demo or Damo (Greek: Δεμώ, Δαμώ; fl. c. AD 200) was a Greek woman of the Roman period, known for a single epigram, engraved upon the Colossus of Memnon, which bears her name. She speaks of herself therein as a lyric poetess dedicated to the Muses, but nothing is known of her life.[1]
Identity
Demo was evidently Greek, as her name, a traditional epithet of Demeter, signifies. The name was relatively common in the Hellenistic world, in Egypt and elsewhere, and she cannot be further identified. The date of her visit to the Colossus of Memnon cannot be established with certainty, but internal evidence on the left leg suggests her poem was inscribed there at some point in or after AD 196.[2]
Epigram
There are a number of graffiti inscriptions on the Colossus of Memnon. Following three epigrams by Julia Balbilla, a fourth epigram, in elegiac couplets, entitled and presumably authored by "Demo" or "Damo" (the Greek inscription is difficult to read), is a dedication to the Muses.[2] The poem is traditionally published with the works of Balbilla, though the internal evidence suggests a different author.[1]
In the poem, Demo explains that Memnon has shown her special respect. In return, Demo offers the gift for poetry, as a gift to the hero. At the end of this epigram, she addresses Memnon, highlighting his divine status by recalling his strength and holiness.[2]
Demo, like Julia Balbilla, writes in the artificial and poetic Aeolic dialect. The language indicates she was knowledgeable in Homeric poetry—'bearing a pleasant gift', for example, alludes to the use of that phrase throughout the Iliad and Odyssey.[a][2] 
"""

In [15]:
# If needed, async is available too: summary = await summarize.invoke_async(input_text)
summary = summarize(input_text)

print(summary)


Demo was a Greek poetess of the Roman period, known for a single epigram inscribed upon the Colossus of Memnon. Her identity is unknown, but her poem suggests she was knowledgeable in Homeric poetry. In the poem, she offers the gift of poetry to the hero Memnon, highlighting his divine status by recalling his strength and holiness.


## basic chat experience with context variable

In [16]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

In [17]:
# Semantic function 
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)

In [18]:
# initializing context
context = kernel.create_new_context()
context["history"] = ""

In [19]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

 Hi there! What kind of books are you looking for? Fiction, non-fiction, biographies, etc.?


In [20]:
# update history with context
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot:  Hi there! What kind of books are you looking for? Fiction, non-fiction, biographies, etc.?



In [21]:
# continue chatting
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [22]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot:  I would recommend Thucydides' History of the Peloponnesian War. It is an excellent source of information about the history and culture of ancient Greece.


In [23]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot:  Thucydides' History of the Peloponnesian War is an account of the war between Athens and Sparta in the 5th century BC. It is an important source of information about the culture and politics of the ancient Greeks.


## Planner - (Equivalent to Chains)

In [24]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [26]:
# providing skills to the planner

from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../semantic-kernel/samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

In [27]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""
original_plan = await planner.create_plan_async(ask, kernel)

In [28]:
print(original_plan.generated_plan)

    {
        "input": "Valentine's Day Date Ideas",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "WriterSkill.Translate", "args": {"language": "French"}},
            {"function": "TextSkill.uppercase"}
        ]
    }


In [29]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

In [30]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [31]:
print(new_plan.generated_plan)

    {
        "input": "Valentine's Day Date Ideas",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "ShakespeareSkill.shakespeare"},
            {"function": "WriterSkill.Translate", "args": {"language": "French"}},
            {"function": "TextSkill.uppercase"}
        ]
    }


In [32]:
# Execute the plan
results = await planner.execute_plan_async(new_plan, kernel)
print(results)

1. FAITES UNE PROMENADE BÉNIE DES ÉTOILES DANS LE PARC
2. FAITES UN BANQUET DANS LA COUR ARRIÈRE
3. ALLEZ AU CINÉMA-MOTEUR
4. FAITES UN DÎNER À LA BOUGIE À LA MAISON
5. FAITES UNE SOIRÉE JEUX AVEC UNE TOUCHE PASSIONNÉE
6. VISITEZ UN MUSÉE DES ARTS
7. PRENEZ DES LEÇONS DE CUISINE ENSEMBLE
8. VISITEZ UNE VIGNOBLE À PROXIMITÉ
9. FAITES UNE AVENTURE EN MONTGOLFIÈRE
10. ALLEZ PATINER SUR UN ÉTANG GELÉ
